<a href="https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras?scriptVersionId=195844539" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<center><h1>Fine-tuning Gemma 2 model using LoRA and Keras</h1></center>

<center><img src="https://res.infoq.com/news/2024/02/google-gemma-open-model/en/headerimage/generatedHeaderImage-1708977571481.jpg" width="400"></center>


# Introduction

This notebook will demonstrate three things:

1. How to fine-tune Gemma model using LoRA
2. Creation of a specialised class to query about Kaggle features
3. Some results of querying about Kaggle Docs

This work is largely based on previous work. Here I list the sources:

1. Gemma 2 Model Card, Kaggle Models,https://www.kaggle.com/models/google/gemma-2/
2. Kaggle QA with Gemma - KerasNLP Starter, Kaggle Code, https://www.kaggle.com/code/awsaf49/kaggle-qa-with-gemma-kerasnlp-starter (Version 11)  
3. Fine-tune Gemma models in Keras using LoRA, Kaggle Code, https://www.kaggle.com/code/nilaychauhan/fine-tune-gemma-models-in-keras-using-lora (Version 1) 
4. Edward J. Hu, Yelong Shen, Phillip Wallis, Zeyuan Allen-Zhu, Yuanzhi Li, Shean Wang, Lu Wang, Weizhu Chen, LoRA: Low-Rank Adaptation of Large Language Models, ArXiv, https://arxiv.org/pdf/2106.09685.pdf
5. Abheesht Sharma, Matthew Watson, Parameter-efficient fine-tuning of GPT-2 with LoRA, https://keras.io/examples/nlp/parameter_efficient_finetuning_of_gpt2_with_lora/
6. Keras 3 API documentation / KerasNLP / Models / Gemma, https://keras.io/api/keras_nlp/models/gemma/
7. Unlock the Power of Gemma 2: Prompt it like a Pro, https://www.kaggle.com/code/gpreda/unlock-the-power-of-gemma-2-prompt-it-like-a-pro  
8. Fine-tune Gemma using LoRA and Keras, https://www.kaggle.com/code/gpreda/fine-tune-gemma-using-lora-and-keras
9. Fine-tunning Gemma model with Kaggle Docs data, https://www.kaggle.com/code/gpreda/fine-tunning-gemma-model-with-kaggle-docs-data
10. Kaggle Docs, Kaggle Dataset, https://www.kaggle.com/datasets/awsaf49/kaggle-docs  


**Let's go**!


# What is Gemma 2?

Gemma is a collection of lightweight, advanced open models developed by Google, leveraging the same research and technology behind the Gemini models. These models are text-to-text, decoder-only large language models available in English, with open weights provided for both pre-trained and instruction-tuned versions. Gemma models excel in a range of text generation tasks, such as question answering, summarization, and reasoning. Their compact size allows for deployment in resource-constrained environments like laptops, desktops, or personal cloud infrastructure, making state-of-the-art AI models more accessible and encouraging innovation for all. 

Gemma 2 represent the 2nd generation of Gemma models. These models were trained on a dataset of text data that includes a wide variety of sources. The **27B** model was trained with **13 trillion** tokens, the **9B** model was trained with **8 trillion tokens**, and **2B** model was trained with **2 trillion** tokens. Here is a summary of their key components: 
* **Web Documents**: A diverse collection of web text ensures the model is exposed to a broad range of linguistic styles, topics, and vocabulary. Primarily English-language content.
* **Code**: Exposing the model to code helps it to learn the syntax and patterns of programming languages, which improves its ability to generate code or understand code-related questions.
* **Mathematics**: Training on mathematical text helps the model learn logical reasoning, symbolic representation, and to address mathematical queries.

To learn more about Gemma 2, follow this link: [Gemma 2 Model Card](https://www.kaggle.com/models/google/gemma-2).




# What is LoRA?  

**LoRA** stands for **Low-Rank Adaptation**. It is a method used to fine-tune large language models (LLMs) by freezing the weights of the LLM and injecting trainable rank-decomposition matrices. The number of trainable parameters during fine-tunning will decrease therefore considerably. According to **LoRA** paper, this number decreases **10,000 times**, and the computational resources size decreases 3 times. 

# How we proceed?

For fine-tunning with LoRA, we will follow the steps:

1. Install prerequisites
2. Load and process the data for fine-tuning
3. Initialize the code for Gemma causal language model (Gemma Causal LM)
4. Perform fine-tuning
5. Test the fine-tunned model with questions from the data used for fine-tuning and with aditional questions

# Prerequisites


## Install packages

We start by installing `keras-nlp` and `keras` packages.

In [1]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3
!pip install -q -U kagglehub --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.5.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.


## Import packages

Now we can import the packages we just installed. We will also install `os`, so that we can set the environment variables needed for keras backend. We will use `jax` as `KERAS_BACKEND`.

Because we want to publish the Model from the Notebook, we also include `kagglehub` and import secrets from `Kaggle App`.

In [2]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.
os.environ["JAX_PLATFORMS"] = ""
import keras
import keras_nlp
import kagglehub


from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["KAGGLE_USERNAME"] = user_secrets.get_secret("kaggle_username")
os.environ["KAGGLE_KEY"] = user_secrets.get_secret("kaggle_key")

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

2024-09-08 19:03:26.117528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-08 19:03:26.117650: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-08 19:03:26.264307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Configurations


We use a `Config` class to group the information needed to control the fine-tuning process:
* random seed 
* dataset path
* preset - name of pretrained Gemma 2
* sequence length - this is the maximum size of input sequence for training
* batch size - size of the input batch in training, x 2 as two GPUs
* lora rank - rank for LoRA, higher means more trainable parameters 
* learning rate used in the train
* epochs - number of epochs for train

In [3]:
class Config:
    seed = 42
    dataset_path = "/kaggle/input/kaggle-docs/questions_answers"
    preset = "gemma2_2b_en" # name of pretrained Gemma 2
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    lora_rank = 4 # rank for LoRA, higher means more trainable parameters
    learning_rate=8e-5 # learning rate used in train
    epochs = 10 # number of epochs to train

Set a random seed for results reproducibility.

In [4]:
keras.utils.set_random_seed(Config.seed)

# Load the data


We load the data we will use for fine-tunining.

In [5]:
df = pd.read_csv(f"{Config.dataset_path}/data.csv")
df.head()

,Question,Answer,Category
0,What are the different types of competitions a...,# Types of Competitions\n\nKaggle Competitions...,competition
1,What are the different competition formats on ...,There are handful of different formats competi...,competition
2,How to join a competition?,"Before you start, navigate to the [Competition...",competition
3,"How to form, manage, and disband teams in a co...",Everyone that competes in a Competition does s...,competition
4,How do I make a submission in a competition?,You will need to submit your model predictions...,competition


Let's check the total number of rows in this dataset.

In [6]:
df.shape[0]

60

For easiness, we will create the following template for QA: 

In [7]:
template = "\n\nCategory:\nkaggle-{Category}\n\nQuestion:\n{Question}\n\nAnswer:\n{Answer}"
df["prompt"] = df.apply(lambda row: template.format(Category=row.Category,
                                                             Question=row.Question,
                                                             Answer=row.Answer), axis=1)
data = df.prompt.tolist()

## Template utility function

In [8]:
def colorize_text(text):
    for word, color in zip(["Category", "Question", "Answer"], ["blue", "red", "green"]):
        text = text.replace(f"\n\n{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

# Specialized class to query Gemma


We define a specialized class to query Gemma. But first, we need to initialize an object of GemmaCausalLM class.

## Initialize the code for Gemma Causal LM

In [9]:
gemma_causal_lm = keras_nlp.models.GemmaCausalLM.from_preset(Config.preset)
gemma_causal_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

## Define the specialized class

Here we define the special class `GemmaQA`. 
in the `__init__` we pass the `GemmaCausalLM` object created before.
The `query` member function uses `GemmaCausalLM` member function `generate` to generate the answer, based on a prompt that includes the category and the question.

In [10]:
class GemmaQA:
    def __init__(self, max_length=512):
        self.max_length = max_length
        self.prompt = template
        self.gemma_causal_lm = gemma_causal_lm
        
    def query(self, category, question):
        response = self.gemma_causal_lm.generate(
            self.prompt.format(
                Category=category,
                Question=question,
                Answer=""), 
            max_length=self.max_length)
        display(Markdown(colorize_text(response)))
        

## Gemma preprocessor


This preprocessing layer will take in batches of strings, and return outputs in a ```(x, y, sample_weight)``` format, where the y label is the next token id in the x sequence.

From the code below, we can see that, after the preprocessor, the data shape is ```(num_samples, sequence_length)```.

In [11]:
x, y, sample_weight = gemma_causal_lm.preprocessor(data[0:2])

In [12]:
print(x, y)

{'token_ids': Array([[   2,  109, 8606, ...,    0,    0,    0],
       [   2,  109, 8606, ...,    0,    0,    0]], dtype=int32), 'padding_mask': Array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False]], dtype=bool)} [[   109   8606 235292 ...      0      0      0]
 [   109   8606 235292 ...      0      0      0]]


# Perform fine-tuning with LoRA

## Enable LoRA for the model

LoRA rank is setting the number of trainable parameters. A larger rank will result in a larger number of parameters to train.

In [13]:
# Enable LoRA for the model and set the LoRA rank to the lora_rank as set in Config (4).
gemma_causal_lm.backbone.enable_lora(rank=Config.lora_rank)
gemma_causal_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

We see that only a small part of the parameters are trainable. 2.6 billions parameters total, and only 2.9 Millions parameters trainable.

## Run the training sequence

We set the `sequence_length` for the `GemmaCausalLM` (from configuration, will be 512).
We compile the model, with the loss, optimizer and metric.
For the metric, it is used `SparseCategoricalAccuracy`. This metric calculates how often predictions match integer labels.

In [14]:
#set sequence length cf. config (512)
gemma_causal_lm.preprocessor.sequence_length = Config.sequence_length 

# Compile the model with loss, optimizer, and metric
gemma_causal_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=Config.learning_rate),
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train model
gemma_causal_lm.fit(data, epochs=Config.epochs, batch_size=Config.batch_size)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 89s 842ms/step - loss: 1.6851 - sparse_categorical_accuracy: 0.5356
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 835ms/step - loss: 1.6007 - sparse_categorical_accuracy: 0.5475
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 835ms/step - loss: 1.5246 - sparse_categorical_accuracy: 0.5575
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 835ms/step - loss: 1.4837 - sparse_categorical_accuracy: 0.5646
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 835ms/step - loss: 1.4438 - sparse_categorical_accuracy: 0.5767
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 835ms/step - loss: 1.4002 - sparse_categorical_accuracy: 0.5853
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 835ms/step - loss: 1.3505 - sparse_categorical_accuracy: 0.5938
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 835ms/step - loss: 1.2935 - sparse_categorical_accuracy: 0.6077
Epoch 9/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 835ms/step - loss: 1.2293 - sparse_categorical_accuracy: 0.6223
Epoch 10/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 836

# Test the fine-tuned model

We instantiate an object of class GemmaQA. Because `gemma_causal_lm` was fine-tuned using LoRA, `gemma_qa` defined here will use the fine-tuned model.

In [15]:
gemma_qa = GemmaQA()

For start, we are testing the model with some of the data from the training set itself.

## Sample 1

In [16]:
row = df.iloc[0]
gemma_qa.query(row.Category,row.Question)



**<font color='blue'>Category:</font>**
kaggle-competition

**<font color='red'>Question:</font>**
What are the different types of competitions available on Kaggle?

**<font color='green'>Answer:</font>**
Kaggle competitions are a great way to get started with machine learning. There are two main types of competitions: public and private.

## Public Competitions

Public competitions are open to anyone and are a great way to get started with Kaggle. These competitions are a great way to learn new skills, meet other data scientists, and collaborate on interesting projects.

Public competitions are typically hosted by Kaggle’s data science team. These competitions are a great way to get a taste of what Kaggle competitions are like.

## Private Competitions

Private competitions are closed competitions hosted by companies or organizations. These competitions are typically invite-only and require a special invitation to participate.

Private competitions are a great way for companies and organizations to host closed competitions. This can be a great way to source new data science talent or to test new machine learning models in a controlled environment.

Some private competitions may be open source. This means that the data and models used in the competition will be made publicly available. This can be a great way to learn new skills and techniques.

Some private competitions may not be open source. This means that the data and models used in the competition will not be made publicly available. This can be a great way to keep the competition fair and to protect the intellectual property of the company or organization hosting the competition.

Some private competitions may be a mix of open source and closed source. This is the most common type of private competition.

Some private competitions may be a mix of open source and closed source. This is the most common type of private competition.

Some private competitions may be closed source. This means that the data and models used in the competition will not be made publicly available. This can be a great way to keep the competition fair and to protect the intellectual property of the company or organization hosting the competition.

Some private competitions may be closed source. This means that the data and models used in the competition will not be made publicly available. This can be a great way to keep the competition fair and to protect the intellectual property of the company or organization hosting the competition.

Some private competitions may be closed source. This means that the data and models used in the competition will not be made publicly available. This can be a great way to keep the competition fair and to protect the intellectual property of the company or organization hosting the competition.

Some private competitions may be

## Sample 2

In [17]:
row = df.iloc[15]
gemma_qa.query(row.Category,row.Question)



**<font color='blue'>Category:</font>**
kaggle-tpu

**<font color='red'>Question:</font>**
How to load and save model on TPU?

**<font color='green'>Answer:</font>**
When using TPU, you can load and save models using the same syntax as on CPU. However, there are a few differences:

- TPU models must be saved in the `keras.models.load_model` format.
- TPU models must be saved in the `keras.models.save_model` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format.
- TPU models must be saved in the `h5` format

## Sample 3

In [18]:
row = df.iloc[25]
gemma_qa.query(row.Category,row.Question)



**<font color='blue'>Category:</font>**
kaggle-noteboook

**<font color='red'>Question:</font>**
What are the different types of notebooks available on Kaggle?

**<font color='green'>Answer:</font>**
Kaggle Notebooks are a powerful tool for data science. They allow you to write code, run analyses, and share your findings in a reproducible and collaborative way. There are two types of Notebooks available on Kaggle: Public Notebooks and Private Notebooks.

## Public Notebooks

Public Notebooks are a great way to share your work with the community. Anyone can view and run your code, but only you can make changes to the notebook.

Public Notebooks are a great way to share your work with the community. Anyone can view and run your code, but only you can make changes to the notebook.

Public Notebooks are a great way to share your work with the community. Anyone can view and run your code, but only you can make changes to the notebook.

Public Notebooks are a great way to share your work with the community. Anyone can view and run your code, but only you can make changes to the notebook.

Public Notebooks are a great way to share your work with the community. Anyone can view and run your code, but only you can make changes to the notebook.

Public Notebooks are a great way to share your work with the community. Anyone can view and run your code, but only you can make changes to the notebook.

Public Notebooks are a great way to share your work with the community. Anyone can view and run your code, but only you can make changes to the notebook.

Public Notebooks are a great way to share your work with the community. Anyone can view and run your code, but only you can make changes to the notebook.

Public Notebooks are a great way to share your work with the community. Anyone can view and run your code, but only you can make changes to the notebook.

Public Notebooks are a great way to share your work with the community. Anyone can view and run your code, but only you can make changes to the notebook.

Public Notebooks are a great way to share your work with the community. Anyone can view and run your code, but only you can make changes to the notebook.

Public Notebooks are a great way to share your work with the community. Anyone can view and run your code, but only you can make changes to the notebook.

Public Notebooks are a great way to share your work with the community. Anyone can view and run your code, but only you can make changes to the notebook

## Not seen question(s)

In [19]:
category = "notebook"
question = "How to run a notebook?"
gemma_qa.query(category,question)



**<font color='blue'>Category:</font>**
kaggle-notebook

**<font color='red'>Question:</font>**
How to run a notebook?

**<font color='green'>Answer:</font>**
You can run a notebook in two ways:

- Directly in the browser: click on the "Run" button (or press alt + R on your keyboard) to execute a single cell or right-click on the notebook and select "Run to <n> to <n> (or Run Selected to <n> to <n>) to execute a batch of cells.
- Locally on your computer: download the notebook as a .ipynb file and open it in Jupiter Notebook on your local machine.

## Running Notebook Code

When you run code cells in a Notebook, the kernel will execute that code, evaluate the output, and display it in the Notebook pane. You can execute code in a cell one cell at a time or multiple cells at once by selecting multiple cells and running all of them at once (Ctrl/Cmd-E-R).

You can also run a Notebook in debug mode. Debug mode is useful when you are trying to track down an error in your code. When in debug mode, the Notebook will stop at any syntax errors in the code. You can enable debug mode by clicking on the "Debug" button on the toolbar.

## Running Notebook Code Locally

You can download a Notebook as a .ipynb file and run it locally on your computer. This can be useful for editing and saving your work, or for sharing a Notebook with collaborators.

To download a Notebook locally, click on the "Download" button on the top bar. This will save a copy of the Notebook in your Downloads folder.

To open the Notebook locally, you will need to have Jupiter Notebook installed on your computer. You can download it for free from https://github.com/rfast/r-help/issues#start-here.

Once you have Jupiter Notebook installed, open a terminal or command prompt and navigate to the folder where you saved your .ipynb file. Then, run the command "jupyter notebook". This will open the Notebook editor and load the Notebook you downloaded locally.

## Running Notebooks in Parallel

You can run multiple instances of the kernel simultaneously to execute multiple cells in parallel. This can be useful for speeding up the execution of a Notebook with a large number of cells.

To run multiple instances of the kernel, click on the "Run" button on the top bar and select "Run to <n> to <n> (or Run Selected to

In [20]:
category = "discussions"
question = "How to create a discussion topic?"
gemma_qa.query(category,question)



**<font color='blue'>Category:</font>**
kaggle-discussions

**<font color='red'>Question:</font>**
How to create a discussion topic?

**<font color='green'>Answer:</font>**
Discussions are a great way to ask questions, share ideas, and learn from others in the Kaggle community.

To start a discussion, click on the "Discussions" tab in the menu bar at the top of the page. Then click on the "New Discussion" button.

You will be prompted to enter a title and description for your discussion.

## Title

Your title should be descriptive and clear. It should also be unique. If you’re starting a discussion about a specific dataset, for example, make sure the title is not the same as a previous discussion about the same dataset.

## Description

Your description is a great place to provide more context about your discussion. You can also use the description to link to a relevant notebook or dataset.

## Tags

Tags are keywords that help other users find discussions that are relevant to their interests. You can add up to 5 tags to your discussion.

To add tags, click on the "Tags" field and start typing. You will see a list of suggested tags. Click and drag to select the tags you want.

## Participants

Discussions are public forums. Anyone can view and contribute to a discussion.

However, you can also choose to make the discussion private. This way only people with a link to the discussion will be able to see or contribute to it.

To make a discussion private, click on the "Participants" button and select "Private".

## Creating a Discussion

Now you’re ready to start a discussion!

Click on the “Discussions” tab in the menu bar at the top of the page. Then click on the “New Discussion” button.

You will be prompted to enter a title and description for your discussion.

## Title

Your title should be descriptive and clear. It should also be unique. If you’re starting a discussion about a specific dataset, for example, make sure the title is not the same as a previous discussion about the same dataset.

## Description

Your description is a great place to provide more context about your discussion. You can also use the description to link to a relevant notebook or dataset.

## Tags

Tags are keywords that help other users find discussions that are relevant to their interests. You can add up to 5 tags to your discussion.

To add tags, click on the “Tags” field and start typing. You will see a list of

In [21]:
category = "competitions"
question = "What is a code competition?"
gemma_qa.query(category,question)



**<font color='blue'>Category:</font>**
kaggle-competitions

**<font color='red'>Question:</font>**
What is a code competition?

**<font color='green'>Answer:</font>**
Code competitions are a great way to practice your skills and learn new techniques. They are also a great way to get your work in front of the Kaggle community.

There are two types of code competitions:
1. Kaggle Notebooks Competitions: These competitions are run on Kaggle Notebooks. You will be provided with a dataset and a set of tasks to complete.
2. Code Challenges: These competitions are run on Kaggle’s public Datasets. You will be provided with a dataset and a set of tasks to complete.

Kaggle Notebooks Competitions are a great way to get started with code competitions. They are relatively easy to understand and follow, and the datasets are often well-suited for beginners.

Code Challenges are a more advanced type of code competition. You will need to be familiar with more advanced techniques, such as using machine learning models, to succeed.

## Kaggle Notebooks Competitions

Kaggle Notebooks Competitions are a great way to get started with code competitions. They are relatively easy to understand and follow, and the datasets are often well-suited for beginners.

There are two types of Kaggle Notebooks Competitions:

- **Public Competitions**: Competitions that are open to the public.
- **Invite-Only Competitions**: Competitions that are only accessible to people who have been invited by the competition organizer.

Invite-Only Competitions are typically more challenging than Public Competitions, as they are designed for a more advanced audience.

If you’re new to code competitions, we recommend starting with a Public Competition.

## Code Challenges

Code Challenges are a more advanced type of code competition. You will need to be familiar with more advanced techniques, such as using machine learning models, to succeed.

There are two types of Code Challenges:

- **Public Competitions**: Competitions that are open to the public.
- **Invite-Only Competitions**: Competitions that are only accessible to people who have been invited by the competition organizer.

Invite-Only Competitions are typically more challenging than Public Competitions, as they are designed for a more advanced audience.

If you’re new to code competitions, we recommend starting with a Public Competition.

## How to Win

There are a few key things to keep in mind if you want to win a code competition:

- **Read the instructions carefully**: The competition organizers will provide instructions on how to complete the tasks. Make sure you understand them before starting.


In [22]:
category = "datasets"
question = "What are the steps to create a Kaggle dataset?"
gemma_qa.query(category,question)



**<font color='blue'>Category:</font>**
kaggle-datasets

**<font color='red'>Question:</font>**
What are the steps to create a Kaggle dataset?

**<font color='green'>Answer:</font>**
To create a Kaggle dataset, follow these steps:

1. Navigate to https://www.kaggle.com/datasets.
2. Click on the "New Dataset" button.
3. Fill out the form with the following information:

- **Dataset Name**: The name of your dataset.
- **Description**: A description of your dataset.
- **License**: A license for your dataset.
- **File Structure**: The file structure of your dataset.
- **Data Files**: The files that make up your dataset.
- **Thumbnail**: A preview image for your dataset.

1. Upload your files.
2. Review your dataset.
3. Publish your dataset.
4. Share your dataset!

You can also create a dataset programmatically using the Kaggle API.

## Creating a Dataset Programmatically

You can create a Kaggle dataset programmatically using the Kaggle API.

To get started, create a new file called `create_dataset.py` and add the following code:

```python
import kaggle as kag

# Replace with your Kaggle API token
kag.Dataset.create(
    "YourDatasetName",
    "YourDatasetDescription",
    "YourFileStructure",
    "YourFileUrls",
    "YourLicense",
    "YourThumbnail",
)
```

For more information and examples, see the [API documentation](https://www.kaggle.com/docs/reference/api/datasets/).

## Creating a Dataset from Scratch

If you don't have a file structure or files to upload, you can create a dataset from scratch.

1. Fill out the form with the following information:

- **Dataset Name**: The name of your dataset.
- **Description**: A description of your dataset.
- **License**: A license for your dataset.
- **File Structure**: The file structure of your dataset.
- **Data Files**: The files that make up your dataset.
- **Thumbnail**: A preview image for your dataset.

1. Click on the "Create Dataset" button.
2. You will be redirected to a new page where you can create the dataset.
3. Review your dataset.
4. Publish your dataset!

## Creating a Dataset from Another Dataset

If you have a

# Save the model

In [23]:
preset_dir = ".\gemma2_2b_en_kaggle_docs"
gemma_causal_lm.save_to_preset(preset_dir)

# Publish Model on Kaggle as a Kaggle Model

We are publishing now the saved model as a Kaggle Model.

In [24]:
kaggle_username = os.environ["KAGGLE_USERNAME"]

kaggle_uri = f"kaggle://{kaggle_username}/gemma2-kaggle-docs/keras/gemma2_2b_en_kaggle_docs"
keras_nlp.upload_preset(kaggle_uri, preset_dir)

Uploading Model https://www.kaggle.com/models/gpreda/gemma2-kaggle-docs/keras/gemma2_2b_en_kaggle_docs ...
Model 'gemma2-kaggle-docs' does not exist or access is forbidden for user 'gpreda'. Creating or handling Model...
Model 'gemma2-kaggle-docs' Created.
Starting upload for file .\gemma2_2b_en_kaggle_docs/task.json


Uploading: 100%|██████████| 2.78k/2.78k [00:00<00:00, 4.35kB/s]

Upload successful: .\gemma2_2b_en_kaggle_docs/task.json (3KB)
Starting upload for file .\gemma2_2b_en_kaggle_docs/model.weights.h5



Uploading: 100%|██████████| 10.5G/10.5G [02:37<00:00, 66.4MB/s]

Upload successful: .\gemma2_2b_en_kaggle_docs/model.weights.h5 (10GB)
Starting upload for file .\gemma2_2b_en_kaggle_docs/preprocessor.json


...


Uploading: 100%|██████████| 143/143 [00:00<00:00, 236B/s]

Upload successful: .\gemma2_2b_en_kaggle_docs/metadata.json (143B)
Starting upload for file .\gemma2_2b_en_kaggle_docs/assets/tokenizer/vocabulary.spm



Uploading: 100%|██████████| 4.24M/4.24M [00:00<00:00, 5.99MB/s]

Upload successful: .\gemma2_2b_en_kaggle_docs/assets/tokenizer/vocabulary.spm (4MB)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/gpreda/gemma2-kaggle-docs/keras/gemma2_2b_en_kaggle_docs


# Conclusions



We demonstated how to fine-tune a **Gemma 2** model using LoRA.   
We also created a class to run queries to the **Gemma 2** model and tested it with some examples from the existing training data but also with some new, not seen questions.   
We also saved the models as a Keras model. 
Then we published the model as a Kaggle Model on Kaggle Models platform.